<a href="https://colab.research.google.com/github/juhee3199/Machine-learning_advanced-study/blob/master/basic/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import sklearn
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

## K-fold 교차 검증

In [3]:
from sklearn.model_selection import  KFold

iris = load_iris()
features = iris.data
label = iris.target

#dt_clf = DecisionTreeClassifier(random_state=156)

In [7]:
# 5개의 폴드셋으로 분리하는 Kfold 객체 생성

kfold = KFold(n_splits=5)
cv_acc = []  # 세트별 정확도를 담을 리스트.

In [14]:
n_iter = 0

# KFold 객체의 split() 객체 호출. 
# 이를 통해 폴드별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환

for train_index, test_index in kfold.split(features):
  # kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
  X_train, X_test = features[train_index], features[test_index]
  y_train, y_test = label[train_index], label[test_index]

  # 학습 및 예측
  dt_clf.fit(X_train, y_train)
  pred = dt_clf.predict(X_test)
  n_iter +=1

  # 반복 시 마다 정확도 측정
  acc = np.round(accuracy_score(y_test, pred), 4)
  train_size = X_train.shape[0]
  test_size = X_test.shape[0]
  print('교차 검증 정확도', acc)

  cv_acc.append(acc)

print('평균 검증 정확도', np.mean(cv_acc))

교차 검증 정확도 1.0
교차 검증 정확도 0.9667
교차 검증 정확도 0.8667
교차 검증 정확도 0.9333
교차 검증 정확도 0.7333
평균 검증 정확도 0.95


## Stratified K fold
- : 불균형한 분포도를 가진 레이블 데이터 집합을 위한 K폴드 방식
  - 불균형한 분포 = 특정 레이블 값이 매우 많거나 적어서 분포가 한쪽으로 치우치는 것


- 일반적으로 '분류'에서의 교차 검증은 K폴드가 아니라 Stratified K 폴드로 분할되야 한다.
- '회귀' 에서는 Stratified K폴드가 지원되지 않는다.
  - 회귀의 target은 이산값 형태의 레이블이 아니라 연속된 숫자값이기 때문에 결정값별로 분포를 정하는 것이 의미가 없음.

In [4]:
import pandas as pd

iris_df = pd.DataFrame(data=iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [5]:
from sklearn.model_selection import StratifiedKFold

dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy = []

# StratifiedKfold의 split 호출 시, 반드시 레이블 데이터 셋도 추가 입력.
# 레이블 데이터의 분포도에 따라 학습/검증 데이터 셋을 나누기 때문에

for train_index, test_index in skfold.split(features, label):
  # 학습용, 검증용 테스트 데이터 추출
  X_train , X_test = features[train_index], features[test_index]
  y_train, y_test = label[train_index], label[test_index]

  # 학습 및 예측
  dt_clf.fit(X_train, y_train)
  pred = dt_clf.predict(X_test)
  
  # 반복 시마다 정확도 측정
  n_iter +=1
  accuracy = np.round(accuracy_score(y_test,pred), 4)
  train_size = X_train.shape[0]
  test_size = X_test.shape[0]

  print('교차 검증 정확도', accuracy)
  cv_accuracy.append(accuracy)


# 교차 검증별 정확도 및 평균 정확도 계산 
print(' 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print(' 평균 검증 정확도:', np.mean(cv_accuracy))

교차 검증 정확도 0.98
교차 검증 정확도 0.94
교차 검증 정확도 0.98
 교차 검증별 정확도: [0.98 0.94 0.98]
 평균 검증 정확도: 0.9666666666666667


## cross_val_score()

- 간편한 교차 검증
- cross_val_score(estimator, X, y=None, scoring=None, cv=None, n_jobs=1, verbose=0, fit_params=None, pre_dispatch='2*n_jobs')
  - 주요 파라미터: estimator, X, y, scoring, cv

- estimator로 classifier가 입력되면 stratifiedKfold가 적용되고, regression이 입력되면 KFold로 분활된다.


In [6]:
from sklearn.model_selection import cross_val_score

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label  = iris_data.target

scores = cross_val_score(dt_clf, data, label, scoring = 'accuracy', cv = 3)
print(np.round(scores, 4))
print(np.round(np.mean(scores), 4))


[0.98 0.94 0.98]
0.9667


## GridCV
 - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에 수행

- 주요 파라미터: estimator, param_grid, scoring, cv, refit(default=True, 최적의 하이퍼파라미터로 학습)


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

iris_data = load_iris()
x_train, x_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.2, random_state=121)

dtree= DecisionTreeClassifier()

# 하이퍼 파라미터
params = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

grid_dtree  = GridSearchCV(dtree, param_grid = params, cv=3, refit=True)

grid_dtree.fit(x_train, y_train)

# GridSearchCV 결과를 DF로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score','split0_test_score','split1_test_score', 'split2_test_score']]
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000514,0.000071,0.000363,0.000060,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.700,0.7,0.70,0.700000,1.110223e-16,5
1,0.000364,0.000004,0.000289,0.000047,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.700,0.7,0.70,0.700000,1.110223e-16,5
2,0.000350,0.000014,0.000234,0.000004,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.925,1.0,0.95,0.958333,3.118048e-02,3
3,0.000366,0.000024,0.000285,0.000034,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.925,1.0,0.95,0.958333,3.118048e-02,3
4,0.000429,0.000073,0.000319,0.000042,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.975,1.0,0.95,0.975000,2.041241e-02,1
5,0.000415,0.000064,0.000300,0.000042,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.975,1.0,0.95,0.975000,2.041241e-02,1


In [15]:
print('최적의 하이퍼 파라미터: ', grid_dtree.best_params_)
print('최고 정확도 ', grid_dtree.best_score_)

최적의 하이퍼 파라미터:  {'max_depth': 3, 'min_samples_split': 2}
최고 정확도  0.975


In [16]:
# GridsearchCV로 학습된 estimator 반환

estimator = grid_dtree.best_estimator_
pred = estimator.predict(x_test)
print('테스트 데이터 정확도: ', accuracy_score(y_test, pred))

테스트 데이터 정확도:  0.9666666666666667
